BASIC LLM

In [ ]:
import os

In [ ]:
from getpass import getpass
os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")


Enter your Google API key: ··········


In [ ]:


# Install necessary packages
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain-google-genai

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Function to load the LLM model
def load_llm(model="gemini-1.5-pro"):
    if model == "gemini-1.5-pro":
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
        return llm
    elif model == "gemini-1.5-flash":
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
        return llm
    else:
        raise ValueError("Invalid model name")

# Function to get the prompt template
def get_prompt_template():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Write a concise summary of the following in {num_words} words:\n\n",
            ),
            ("human", "{context}")
        ]
    )
    return prompt

# Function to summarize text
def summarize_text(text, num_words=50, model="gemini-1.5-pro"):
    # Load LLM
    llm = load_llm(model)

    # Get Prompt Template
    prompt = get_prompt_template()

    # Create chain
    chain = prompt | llm

    # Run the chain with input data
    result = chain.invoke({
        "context": text,
        "num_words": num_words
    })

    # Return the result content
    return result.content

# Example text to summarize
text = '''
Text summarization involves creating a summary of a source text using natural language processing.
This is useful for condensing long-form text, audio, or video into a shorter, more digestible form that still conveys the main points.
Examples include news articles, scientific papers, podcasts, speeches, lectures, and meeting recordings.

There are two main types of summarization:

Extractive summarization: This type identifies and extracts key phrases or sentences (i.e., excerpts) from the source text and combines them into a summary.
It leaves the original text unchanged and only selects the important parts.

Abstractive summarization: This type involves understanding the main ideas in the source text and creating a new summary that expresses those ideas in a fresh and condensed way (i.e., paraphrasing).
It's more complex because it requires a deeper understanding of the source text and the ability to convey the same information in fewer words.
'''

# Specify the number of words for the summary and the model
summary = summarize_text(text, num_words=50, model="gemini-1.5-flash")

# Print the original text and the summary
print(f"Original Text: {text}")
print(f"\nText Length: {len(text)}")
print("=" * 100)
print(f"\nSummary: {summary}")
print(f"\nSummary Length: {len(summary)}")


Original Text: 
Text summarization involves creating a summary of a source text using natural language processing.
This is useful for condensing long-form text, audio, or video into a shorter, more digestible form that still conveys the main points.
Examples include news articles, scientific papers, podcasts, speeches, lectures, and meeting recordings.

There are two main types of summarization:

Extractive summarization: This type identifies and extracts key phrases or sentences (i.e., excerpts) from the source text and combines them into a summary.
It leaves the original text unchanged and only selects the important parts.

Abstractive summarization: This type involves understanding the main ideas in the source text and creating a new summary that expresses those ideas in a fresh and condensed way (i.e., paraphrasing).
It's more complex because it requires a deeper understanding of the source text and the ability to convey the same information in fewer words.


Text Length: 961

Summ

MAP REDUCE

In [ ]:
!pip install gradio
!pip install langchain_google_genai
!pip install tiktoken openai langchain --quiet
!pip install langchain-community langchain-google-vertexai beautifulsoup4 langgraph pypdf gradio --quiet
!pip install matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot


In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph
from langchain.document_loaders import PyPDFLoader
import textwrap
from typing import List, Literal, TypedDict

In [ ]:
import os

In [ ]:
from getpass import getpass
os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [ ]:
documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yellow", metadata={"title": "banana_book"}),
]

In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print("Summary:", wrapped_text)

Summary: Fruits have various colors.


ITERATIVE REFINEMENT

In [ ]:
class State(TypedDict):
    contents: List[str]
    index: int
    summary: str

summarize_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following: {context}")])
initial_summary_chain = summarize_prompt | llm | StrOutputParser()

refine_template = """
Produce a final summary.
Existing summary up to this point:
{existing_answer}
New context:
------------
{context}
------------
Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_summary_chain = refine_prompt | llm | StrOutputParser()

async def generate_initial_summary(state: State, config):
    summary = await initial_summary_chain.ainvoke(state["contents"][0], config)
    return {"summary": summary, "index": 1}

async def refine_summary(state: State, config):
    content = state["contents"][state["index"]]
    summary = await refine_summary_chain.ainvoke({"existing_answer": state["summary"], "context": content}, config)
    return {"summary": summary, "index": state["index"] + 1}

def should_refine(state: State) -> Literal["refine_summary", END]:
    return "refine_summary" if state["index"] < len(state["contents"]) else END

graph = StateGraph(State)
graph.add_node("generate_initial_summary", generate_initial_summary)
graph.add_node("refine_summary", refine_summary)
graph.add_edge(START, "generate_initial_summary")
graph.add_conditional_edges("generate_initial_summary", should_refine)
graph.add_conditional_edges("refine_summary", should_refine)
app = graph.compile()

async for step in app.astream({"contents": [doc.page_content for doc in documents]}, stream_mode="values"):
    if summary := step.get("summary"):
        print("Summary:", summary)


Summary: Apples can be red.

Summary: Apples can be red, and blueberries are blue.

Summary: Apples are red, blueberries are blue, and bananas are yellow.



UPLOADING PDF

In [ ]:
!pip install -U gradio langchain langchain-community pypdf langchain-google-genai langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 

In [ ]:
import os

In [ ]:
from getpass import getpass
os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
import gradio as gr

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
      model="gemini-1.5-flash",
      temperature=0,
      max_tokens=None,
      timeout=None,
      max_retries=2
)

In [ ]:
def summarize_pdf(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)
    docs = loader.load_and_split()
    chain = load_summarize_chain(llm, chain_type="refine") #map_reduce
    summary = chain.invoke(docs)

    return summary['output_text']

In [ ]:
# Upload the file
from google.colab import files
uploaded = files.upload()

file_path = "Infra.pdf"  # Use the file name from the upload

summary = summarize_pdf(file_path)

Saving Infra.pdf to Infra.pdf


In [ ]:
print(type(summary))

<class 'str'>


In [ ]:
#Summary
print(summary)

Sri Manakula Vinayagar Engineering College's AI and Data Science department boasts five classrooms (65 student capacity each), equipped with projectors, greenboards, ergonomic seating, natural light, LED lighting, ceiling fans, and an emergency alarm system.  Three faculty rooms offer ergonomic desks, chairs, personal storage, and collaborative spaces.  Two clean restrooms and a centrally located lift (with backup power) ensure accessibility.  The department also features clearly marked emergency exits, regularly scheduled safety drills, and a clear display of capacity limits in each room.  This infrastructure is designed to meet the highest standards of academic excellence, providing a safe, comfortable, and conducive learning environment.

